In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd

In [3]:
driver = webdriver.Chrome("../../../chromedriver")

In [4]:
def _find_element_click(
    driver,
    by,
    expression,
):
    """Find the element and click then  handle all type of exception during click
    Args:
        driver (selenium.driver): Selenium driver
        by (selenium.webdriver.common.by): Type of selector
            (By.XPATH, By.CSS_SELECTOR ...)
        expression (str): Selector expression to the element to click on
    Returns:
        bool: True if successfully clicked on the element
    """
    end_time = time.time() + 30
    while True:
        try:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
            time.sleep(2)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
            web_element = driver.find_element(by=by, value=expression)
            web_element.click()
            return True
        except:
            if time.time() > end_time:
                time.sleep(4)
                break
    return False

In [4]:
url = "https://www.tripadvisor.com/Airline_Review-d8728986-Reviews-Aer-Lingus"
driver.get(url)
page = BeautifulSoup(driver.page_source)

In [16]:
#Go through all the pages of reviews
i = 0
airline = []
reviews = []
dates = []
scores = []
tags = []

for airline in airlines["airlines"]:
    url = "https://www.tripadvisor.com" + airlines["airlines" == airline]["links"]
    try:
        total_pages = int(page.find("div", {"class": "pageNumbers"}).find_all("a")[-1].text)
    except:
        total_pages = 1
    while i < 10:
        page = BeautifulSoup(driver.page_source)

        #Review text
        for review in page.find_all("q", {"class": "IRsGHoPm"}):
            reviews.append(review.text)
        
        #Review date
        for date in page.find_all("span", {"class": "_34Xs-BQm"}):
            dates.append(date.text)

        #Review scores
        for score in page.find_all("div", {"class": "nf9vGX55"}):
            scores.append(int(score.find("span").get("class")[1][7]))

        #Trip information
        for tag in page.find_all("div", {"class": "hpZJCN7D"}):
            tags.append([stag.text for stag in tag.find_all("div", {"class": "_3tp-5a1G"})])
        
        result = _find_element_click(driver, webdriver.common.by.By.XPATH, ".//a[@class='ui_button nav next primary ']")
        if result == False:
            i = total_pages
            print("Failure to go to next page")

        i += 1

In [17]:
reviews = pd.DataFrame({"reviews": reviews, "scores": scores, "tags": tags})
reviews.to_csv("../data/trip_advisor_reviews.csv", sep = ",", index = False)